In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import cv2
from PIL import Image
from sklearn.metrics import accuracy_score

In [2]:
train_path = './Dataset_2/Train/'
test_path = 'Dataset_2/Test/'
data_dir = './Dataset_2'

train_lable = 'Dataset_2/Train.csv'
test_lable = 'Dataset_2/Test.csv'

In [3]:
NUM_CATEGORIES = len(os.listdir(train_path))
NAME_CATEGORIES = os.listdir(train_path)
print(NUM_CATEGORIES)
print(NAME_CATEGORIES)

43
['12', '16', '19', '32', '28', '30', '40', '24', '4', '2', '14', '38', '9', '20', '3', '41', '7', '0', '17', '39', '34', '22', '33', '8', '42', '1', '5', '27', '31', '26', '10', '25', '35', '6', '36', '15', '37', '23', '21', '18', '11', '29', '13']


In [4]:
# Resizing the images to 30x30x3
IMG_HEIGHT = 30
IMG_WIDTH = 30
channels = 3

In [5]:
# move resize
image_data = []
image_labels = []

for i in range(NUM_CATEGORIES):
    path = data_dir + '/Train/' + str(i)
    images = os.listdir(path)
    for img in images:
        try:
            image = cv2.imread(path + '/' + img)
            image_fromarray = Image.fromarray(image, 'RGB')
            #列表不够整齐，如果创建多维列表，子列表中的元素得个数一致，如果不一致，那么默认还是一维列表，把子列表当做元素而已
            resize_image = image_fromarray.resize((IMG_HEIGHT, IMG_WIDTH))      
            imgray = cv2.cvtColor(np.array(resize_image), cv2.COLOR_BGR2GRAY)
            flat_arr=imgray.ravel()
            image_data.append(flat_arr)
            image_labels.append(i)
        except:
            print("Error in " + img)

In [6]:
test = pd.read_csv(data_dir + '/Test.csv')
labels = test["ClassId"].values
imgs = test["Path"].values
test_data =[]
for img in imgs:
    try:
        image = cv2.imread(data_dir + '/' +img)
        image_fromarray = Image.fromarray(image, 'RGB')
        resize_image = image_fromarray.resize((IMG_HEIGHT, IMG_WIDTH))
        imgray = cv2.cvtColor(np.array(resize_image), cv2.COLOR_BGR2GRAY)
        flat_arr=imgray.ravel()
        test_data.append(np.array(flat_arr))
    except:
        print("Error in " + img)

In [7]:
X_test = np.array(test_data)
X_test = X_test/255

In [8]:
img_dict={"images":np.array(imgray),"data":np.array(image_data),"target":np.array(image_labels)}
train = img_dict['data']/255

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, img_dict['target'], test_size=0.3, random_state=0, shuffle=True)

# RandomForest

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

dtc = DecisionTreeClassifier(random_state=0)
rfc = RandomForestClassifier(random_state=0)
dtc = dtc.fit(X_train,y_train)
rfc = rfc.fit(X_train,y_train)
score_c = dtc.score(X_val,y_val)
score_r = rfc.score(X_val,y_val)
print("Single Tree:{}".format(score_c),"Random Forest:{}".format(score_r))

Single Tree:0.7500637592450905 Random Forest:0.9729660800816118


In [11]:
dtc_pred = dtc.predict(X_test)
rf_pred = rfc.predict(X_test)

#Accuracy with the test data
print('Single Tree Test Data accuracy: ',accuracy_score(labels,dtc_pred))
print('Randomforest Test Data accuracy: ',accuracy_score(labels,rf_pred))

Single Tree Test Data accuracy:  0.5429928741092637
Randomforest Test Data accuracy:  0.7566112430720506


# GaussianNB

In [12]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB().fit(X_train,y_train)
score = gnb.score(X_val,y_val)
print('准确率：',score)

准确率： 0.12173765195953413


In [13]:
gnb_pred = gnb.predict(X_test)
print('Gaussian naive bayes Test Data accuracy: ',accuracy_score(labels,gnb_pred))

Gaussian naive bayes Test Data accuracy:  0.10577988915281077


# KNeighborsClassifier

In [18]:
from sklearn.neighbors import KNeighborsClassifier

n_neighbors = [1,3,5,7,9]
for k in n_neighbors:
    knn=KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train,y_train)
    prediction=knn.predict(X_val)
    print("k = {}, acc = {}".format(k,accuracy_score(y_val,prediction)))

k = 1, acc = 0.9060613788999405
k = 3, acc = 0.860154722434753
k = 5, acc = 0.8301453710788065
k = 7, acc = 0.8030264388336309
k = 9, acc = 0.7806681968885488


In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
parameter = {'n_neighbors':[1,3,5,7,9,11]}
knn = KNeighborsClassifier()
knn_clf = GridSearchCV(knn,parameter,cv=5)
knn_clf.fit(X_train,y_train)
print(f'评估最合适的K值为：{(knn_clf.best_params_)["n_neighbors"]}',"其准确率为：%.2f"%knn_clf.best_score_)

评估最合适的K值为：1 其准确率为：0.89


In [21]:
knn=KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train,y_train)
knn_pred = knn.predict(X_test)
print('KNN Test Data accuracy: ',accuracy_score(labels,knn_pred))

KNN Test Data accuracy:  0.37094220110847187


# SVM

In [ ]:
from sklearn.svm import SVC
svc = SVC(kernel="linear",random_state=0)
clf.fit(X_train, y_train)#训练
y_predict = clf.predict(X_test)#预测
print(accuracy_score(y_test, y_predict))#评分